In [1]:
##################################################################################
##### Define all parameters for model tuning
##################################################################################

n_fold = 5
expName = "NT_Site_PredNTS_Classification_DLNN_KgapRFE"
outPath = "Results"
foldName = "folds.pickle"

epochs = 200
batch_size = 16
shuffle = True
seed = None

input_data_folder = "PredNTS_MathFeature_ENC"

monitor = 'val_accuracy'

sub_feature_count = 200

In [2]:
kgap_max = 4

train_data_filename = 'Training-datasets-PredNTS_kgap_{}.csv'
indpe_data_filename = 'independent-dataset-PredNTS_kgap_{}.csv'

In [3]:
import os 
import pickle
import numpy as np
import pandas as pd

import tensorflow as tf

from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve, auc, accuracy_score, precision_score, confusion_matrix
from sklearn.metrics import roc_auc_score, classification_report, matthews_corrcoef

from sklearn.feature_selection import RFE

import math

In [4]:
# print(tf.test.is_gpu_available(cuda_only=True))
# physical_devices = tf.config.experimental.list_physical_devices('GPU')
physical_devices = tf.config.list_physical_devices('GPU')
print(physical_devices)
tf.config.experimental.set_memory_growth(physical_devices[0], True)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [5]:
# ##################################################################################
# ##### define all CUSTOM functions
# ##################################################################################

# def one_hot_encode_nt(sequence, char_dict):
    
#     seq_encoded = np.zeros((len(sequence),len(char_dict)))
    
#     i = 0
#     for single_character in sequence:
#         if(single_character.upper() in char_dict.keys()):
#             seq_encoded[i][char_dict[single_character.upper()]] = 1
#             i = i+1
#         else:
#             raise ValueError('Incorrect character in NT sequence: '+sequence)
#     return seq_encoded

In [6]:
##################################################################################
##### Build k-fold functions
##################################################################################

## Build the K-fold from dataset
def build_kfold(features, labels, k=10, shuffle=False, seed=None):
    
    skf = StratifiedKFold(n_splits=k, shuffle=shuffle, random_state=seed)
    kfoldList = []
    for train_index, test_index in skf.split(features, labels):
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        kfoldList.append({
            "X_train": X_train,
            "X_test": X_test,
            "y_train":y_train,
            "y_test":y_test
        })
    return kfoldList

In [7]:
##################################################################################
##### define evaluator functions
##################################################################################

def pred2label(y_pred):
    y_pred = np.round(y_pred)
    return y_pred

In [8]:
##################################################################################
##### Function to customize the DLNN architecture with parameters
##################################################################################

def DLNN_Classifier(input_vec_shape,
                    dense_decode_units = 128, ## Dense layer parameters
                    prob = 0.5, learn_rate = 0.0001, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
    beta = 0.001
    
    input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
    ######################################################################################################
    ########  Classifier  ################################################################################
    ######################################################################################################
    
    y = tf.keras.layers.Dense(dense_decode_units, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta))(input1)
    
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(int(dense_decode_units/2), 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'relu')(y)
    
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(int(dense_decode_units/4), 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'relu')(y)
    
    y = tf.keras.layers.Dropout(prob)(y)
    
    y = tf.keras.layers.Dense(1, 
                              kernel_regularizer = tf.keras.regularizers.l2(beta), 
                              activation = 'sigmoid')(y)

    ## Generate Model from input and output
    model = tf.keras.models.Model(inputs=input1, outputs=y)
    
    ## Compile model
    if(metrics != None):
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                      loss = loss, metrics = metrics)
    else:
        model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
                      loss = loss)

    return model

In [9]:
# epochs = 200
# batch_size = 16

# ##################################################################################
# ##### Function to customize the DLNN architecture with parameters
# ##################################################################################

# def DLNN_Classifier(input_vec_shape,
#                     dense_decode_units = 8, ## Dense layer parameters,
#                     dense_layers = 2,
#                     prob = 0.5, learn_rate = 0.0001, loss = 'binary_crossentropy', metrics = 'accuracy'):
    
#     beta = 0.001
    
#     input1 = tf.keras.layers.Input(shape=input_vec_shape)
    
#     ######################################################################################################
#     ########  Classifier  ################################################################################
#     ######################################################################################################
    
#     y = tf.keras.layers.Dense(dense_decode_units, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta))(input1)
#     y = tf.keras.layers.BatchNormalization()(y)
#     y = tf.keras.layers.GaussianNoise(stddev=0.1)(y)
#     y = tf.keras.layers.Dropout(prob)(y)
    
#     for i in range(1,dense_layers+1):
    
#         y = tf.keras.layers.Dense(int(dense_decode_units/(2**i)), 
#                                   kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                                  )(y)
#         y = tf.keras.layers.BatchNormalization()(y)
#         y = tf.keras.layers.GaussianNoise(stddev=0.1)(y)
#         y = tf.keras.layers.Dropout(prob)(y)
    
#     y = tf.keras.layers.Dense(1, 
#                               kernel_regularizer = tf.keras.regularizers.l2(beta), 
#                               activation = 'sigmoid')(y)

#     ## Generate Model from input and output
#     model = tf.keras.models.Model(inputs=input1, outputs=y)
    
#     ## Compile model
#     if(metrics != None):
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss, metrics = metrics)
#     else:
#         model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate=learn_rate), 
#                       loss = loss)

#     return model

In [10]:
DLNN_Classifier((sub_feature_count)).summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 200)]             0         
                                                                 
 dense (Dense)               (None, 128)               25728     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 32)                2080      
                                                                 
 dropout_2 (Dropout)         (None, 32)                0     

# Train data preparation

In [11]:
##################################################################################
##### Read CSV data
##################################################################################

for i in range(kgap_max+1):
    
    current_train_data_filepath = os.path.join(input_data_folder, train_data_filename.format(i))
    current_train_data = pd.read_csv(current_train_data_filepath, sep=',', header=0)
    current_train_data = current_train_data.drop('label', axis=1)
    
    if i == 0:
        train_data = current_train_data
    else:
        train_data = pd.merge(
            train_data,
            current_train_data,
            how="inner",
            on='nameseq'
        )

train_data['label'] = pd.Series([int(val.split('_')[-2])
                                 for val in train_data['nameseq']])

train_data = train_data.drop('nameseq', axis=1)

train_features = np.array(train_data.drop('label', axis=1))
train_labels = np.array(train_data['label'])
train_labels = train_labels.reshape((train_labels.shape[0], 1))

In [12]:
##################################################################################
##### Recursive feature selection
##################################################################################

model = DecisionTreeClassifier(criterion="gini")

selector = RFE(model, n_features_to_select=sub_feature_count, step=50)
selector = selector.fit(train_features, train_labels)

feature_indices = np.where(selector.ranking_ == 1)[0]

In [13]:
##################################################################################
##### Extract features and labels, create folds
##################################################################################

train_features = train_features[:, feature_indices]

folds = build_kfold(train_features, train_labels, k=n_fold, shuffle=shuffle, seed=seed)

input_vec_shape = train_features[0].shape

## Write the k-fold dataset to file
foldPath = os.path.join(outPath, expName, "{}fold".format(n_fold))
if(not os.path.isdir(foldPath)):
    os.makedirs(foldPath)
pickle.dump(folds, open(os.path.join(foldPath, foldName), "wb"))

# Independent data

In [14]:
##################################################################################
##### Read CSV data
##################################################################################

for i in range(kgap_max+1):

    current_indpe_data_filepath = os.path.join(input_data_folder, indpe_data_filename.format(i))
    current_indpe_data = pd.read_csv(current_indpe_data_filepath, sep=',', header=0)
    current_indpe_data = current_indpe_data.drop('label', axis=1)
    
    if i == 0:
        indpe_data = current_indpe_data
    else:
        indpe_data = pd.merge(
            indpe_data,
            current_indpe_data,
            how="inner",
            on='nameseq'
        )

indpe_data['label'] = pd.Series([int(val.split('_')[-2])
                                 for val in indpe_data['nameseq']])

indpe_data = indpe_data.drop('nameseq', axis=1)

##################################################################################
##### Extract features and labels, create folds
##################################################################################

indpe_features = np.array(indpe_data.drop('label', axis=1))
indpe_features = indpe_features[:, feature_indices]

indpe_labels = np.array(indpe_data['label'])
indpe_labels = indpe_labels.reshape((indpe_labels.shape[0], 1))

# Training

In [15]:
##################################################################################
##### For each input file, train model and generate different outputs in a structured folder
##################################################################################

## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Train/Test model on all folds, generate evaluations
##################################################################################

## Create and set directory to save model
modelPath = os.path.join(outPath, expName, "{}fold".format(n_fold), "models")
if(not os.path.isdir(modelPath)):
    os.makedirs(modelPath)

i = -1
for fold in folds:
    i += 1
    
    print("\nTrain/Test model on Fold #"+str(i)+".")
    
    model = DLNN_Classifier(input_vec_shape = input_vec_shape)
    
    ## Define the model callbacks for early stopping and saving the model. Then train model
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    modelCallbacks = [
        tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                           monitor = monitor, verbose = 1, save_best_only = True, 
                                           save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
    ]
    
    # adding random shuffling of the dataset for training purpose
    index_arr = np.arange(fold["X_train"].shape[0])
    index_arr = np.random.permutation(index_arr)
    
    model.fit(x = fold["X_train"][index_arr], y = fold["y_train"][index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
              callbacks = modelCallbacks, validation_data = (fold["X_test"], fold["y_test"]))
    
    model = tf.keras.models.load_model(current_model_path)
    
    ##################################################################################
    ##### Prediction and metrics for TRAIN dataset
    ##################################################################################

    y_pred = model.predict(fold["X_train"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_train"], label_pred)
    prec = precision_score(fold["y_train"],label_pred)
    mcc = matthews_corrcoef(fold["y_train"], label_pred)

    conf = confusion_matrix(fold["y_train"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_train"], y_pred)
    auc = roc_auc_score(fold["y_train"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Train")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    ##################################################################################
    ##### Prediction and metrics for TEST dataset
    ##################################################################################

    y_pred = model.predict(fold["X_test"])
    label_pred = pred2label(y_pred)
    
    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(fold["y_test"], label_pred)
    prec = precision_score(fold["y_test"],label_pred)
    mcc = matthews_corrcoef(fold["y_test"], label_pred)

    conf = confusion_matrix(fold["y_test"], label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(fold["y_test"], y_pred)
    auc = roc_auc_score(fold["y_test"], y_pred)
    
    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Test")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    del model
    tf.keras.backend.clear_session()


Train/Test model on Fold #0.
Epoch 1/200
107/120 [=========================>....] - ETA: 0s - loss: 1.0578 - accuracy: 0.5123
Epoch 1: val_loss improved from inf to 0.96763, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 2s 5ms/step - loss: 1.0540 - accuracy: 0.5113 - val_loss: 0.9676 - val_accuracy: 0.5451
Epoch 2/200
109/120 [==========================>...] - ETA: 0s - loss: 1.0304 - accuracy: 0.5304
Epoch 2: val_loss improved from 0.96763 to 0.95796, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 1s 4ms/step - loss: 1.0313 - accuracy: 0.5270 - val_loss: 0.9580 - val_accuracy: 0.5891
Epoch 3/200
107/120 [=========================>....] - ETA: 0s - loss: 1.0154 - accuracy: 0.5426
Epoch 3: val_loss improved from 0.95796 to 0.95065, saving model to Results\NT_Site_PredNTS_Classification_DLNN_K

120/120 [==============================] - 1s 4ms/step - loss: 0.8597 - accuracy: 0.6656 - val_loss: 0.8293 - val_accuracy: 0.7023
Epoch 23/200
110/120 [==========================>...] - ETA: 0s - loss: 0.8286 - accuracy: 0.7000
Epoch 23: val_loss improved from 0.82926 to 0.82233, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 1s 4ms/step - loss: 0.8288 - accuracy: 0.7003 - val_loss: 0.8223 - val_accuracy: 0.7023
Epoch 24/200
112/120 [===========================>..] - ETA: 0s - loss: 0.8263 - accuracy: 0.7020
Epoch 24: val_loss improved from 0.82233 to 0.81637, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 0s 4ms/step - loss: 0.8290 - accuracy: 0.7024 - val_loss: 0.8164 - val_accuracy: 0.7023
Epoch 25/200
107/120 [=========================>....] - ETA: 0s - loss: 0.8331 - accuracy: 0.6998
Epo

Epoch 44/200
111/120 [==========================>...] - ETA: 0s - loss: 0.7226 - accuracy: 0.7641
Epoch 44: val_loss improved from 0.76804 to 0.76755, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 0s 4ms/step - loss: 0.7276 - accuracy: 0.7585 - val_loss: 0.7675 - val_accuracy: 0.7044
Epoch 45/200
112/120 [===========================>..] - ETA: 0s - loss: 0.7355 - accuracy: 0.7472
Epoch 45: val_loss improved from 0.76755 to 0.76571, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 0s 4ms/step - loss: 0.7329 - accuracy: 0.7501 - val_loss: 0.7657 - val_accuracy: 0.7107
Epoch 46/200
111/120 [==========================>...] - ETA: 0s - loss: 0.7362 - accuracy: 0.7444
Epoch 46: val_loss improved from 0.76571 to 0.76335, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\

Epoch 66/200
110/120 [==========================>...] - ETA: 0s - loss: 0.6855 - accuracy: 0.7727
Epoch 66: val_loss did not improve from 0.74264
120/120 [==============================] - 0s 4ms/step - loss: 0.6824 - accuracy: 0.7764 - val_loss: 0.7429 - val_accuracy: 0.7044
Epoch 67/200
109/120 [==========================>...] - ETA: 0s - loss: 0.6668 - accuracy: 0.7838
Epoch 67: val_loss improved from 0.74264 to 0.74108, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 1s 4ms/step - loss: 0.6712 - accuracy: 0.7832 - val_loss: 0.7411 - val_accuracy: 0.7044
Epoch 68/200
113/120 [===========================>..] - ETA: 0s - loss: 0.6760 - accuracy: 0.7721
Epoch 68: val_loss improved from 0.74108 to 0.74080, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 1s 5ms/step - loss: 0.6732 - accuracy: 0.77

120/120 [==============================] - 1s 5ms/step - loss: 0.6253 - accuracy: 0.7895 - val_loss: 0.7241 - val_accuracy: 0.7107
Epoch 90/200
119/120 [============================>.] - ETA: 0s - loss: 0.6250 - accuracy: 0.7878
Epoch 90: val_loss did not improve from 0.72405
120/120 [==============================] - 0s 4ms/step - loss: 0.6250 - accuracy: 0.7879 - val_loss: 0.7247 - val_accuracy: 0.7107
Epoch 91/200
113/120 [===========================>..] - ETA: 0s - loss: 0.6220 - accuracy: 0.7992
Epoch 91: val_loss improved from 0.72405 to 0.72286, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\bestModel-fold0.hdf5
120/120 [==============================] - 0s 4ms/step - loss: 0.6208 - accuracy: 0.7990 - val_loss: 0.7229 - val_accuracy: 0.7107
Epoch 92/200
108/120 [==========================>...] - ETA: 0s - loss: 0.6261 - accuracy: 0.7841
Epoch 92: val_loss did not improve from 0.72286
120/120 [==============================] - 0s 4ms/step - loss:

...

Epoch 81/200
115/120 [===========================>..] - ETA: 0s - loss: 0.6631 - accuracy: 0.7701
Epoch 81: val_loss improved from 0.67843 to 0.67829, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\bestModel-fold4.hdf5
120/120 [==============================] - 1s 4ms/step - loss: 0.6599 - accuracy: 0.7718 - val_loss: 0.6783 - val_accuracy: 0.7626
Epoch 82/200
115/120 [===========================>..] - ETA: 0s - loss: 0.6431 - accuracy: 0.7772
Epoch 82: val_loss improved from 0.67829 to 0.67749, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\bestModel-fold4.hdf5
120/120 [==============================] - 1s 4ms/step - loss: 0.6441 - accuracy: 0.7770 - val_loss: 0.6775 - val_accuracy: 0.7584
Epoch 83/200
118/120 [============================>.] - ETA: 0s - loss: 0.6474 - accuracy: 0.7749
Epoch 83: val_loss improved from 0.67749 to 0.67661, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\

117/120 [============================>.] - ETA: 0s - loss: 0.6082 - accuracy: 0.7885
Epoch 103: val_loss improved from 0.65797 to 0.65762, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\bestModel-fold4.hdf5
120/120 [==============================] - 1s 4ms/step - loss: 0.6080 - accuracy: 0.7870 - val_loss: 0.6576 - val_accuracy: 0.7626
Epoch 104/200
111/120 [==========================>...] - ETA: 0s - loss: 0.6008 - accuracy: 0.7900
Epoch 104: val_loss improved from 0.65762 to 0.65594, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\bestModel-fold4.hdf5
120/120 [==============================] - 1s 5ms/step - loss: 0.5945 - accuracy: 0.7959 - val_loss: 0.6559 - val_accuracy: 0.7647
Epoch 105/200
116/120 [============================>.] - ETA: 0s - loss: 0.5940 - accuracy: 0.7899
Epoch 105: val_loss improved from 0.65594 to 0.65556, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\bestMode

120/120 [==============================] - 0s 4ms/step - loss: 0.5594 - accuracy: 0.8075 - val_loss: 0.6484 - val_accuracy: 0.7563
Epoch 128/200
118/120 [============================>.] - ETA: 0s - loss: 0.5564 - accuracy: 0.7993
Epoch 128: val_loss did not improve from 0.64838
120/120 [==============================] - 0s 4ms/step - loss: 0.5587 - accuracy: 0.7985 - val_loss: 0.6492 - val_accuracy: 0.7542
Epoch 129/200
117/120 [============================>.] - ETA: 0s - loss: 0.5588 - accuracy: 0.7997
Epoch 129: val_loss improved from 0.64838 to 0.64737, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\bestModel-fold4.hdf5
120/120 [==============================] - 1s 4ms/step - loss: 0.5602 - accuracy: 0.7991 - val_loss: 0.6474 - val_accuracy: 0.7521
Epoch 130/200
119/120 [============================>.] - ETA: 0s - loss: 0.5549 - accuracy: 0.8051
Epoch 130: val_loss improved from 0.64737 to 0.64572, saving model to Results\NT_Site_PredNTS_Classificat

120/120 [==============================] - 1s 4ms/step - loss: 0.5122 - accuracy: 0.8353 - val_loss: 0.6439 - val_accuracy: 0.7542
Epoch 156/200
119/120 [============================>.] - ETA: 0s - loss: 0.5108 - accuracy: 0.8220
Epoch 156: val_loss did not improve from 0.64304
120/120 [==============================] - 0s 4ms/step - loss: 0.5106 - accuracy: 0.8221 - val_loss: 0.6465 - val_accuracy: 0.7542
Epoch 157/200
118/120 [============================>.] - ETA: 0s - loss: 0.5092 - accuracy: 0.8257
Epoch 157: val_loss did not improve from 0.64304
120/120 [==============================] - 0s 4ms/step - loss: 0.5100 - accuracy: 0.8248 - val_loss: 0.6462 - val_accuracy: 0.7584
Epoch 158/200
117/120 [============================>.] - ETA: 0s - loss: 0.5123 - accuracy: 0.8328
Epoch 158: val_loss did not improve from 0.64304
120/120 [==============================] - 0s 4ms/step - loss: 0.5112 - accuracy: 0.8326 - val_loss: 0.6460 - val_accuracy: 0.7584
Epoch 159/200
113/120 [=========

Epoch 185/200
118/120 [============================>.] - ETA: 0s - loss: 0.4652 - accuracy: 0.8416
Epoch 185: val_loss did not improve from 0.64304
120/120 [==============================] - 0s 4ms/step - loss: 0.4651 - accuracy: 0.8421 - val_loss: 0.6522 - val_accuracy: 0.7773
Epoch 186/200
117/120 [============================>.] - ETA: 0s - loss: 0.4629 - accuracy: 0.8478
Epoch 186: val_loss did not improve from 0.64304
120/120 [==============================] - 0s 4ms/step - loss: 0.4626 - accuracy: 0.8478 - val_loss: 0.6504 - val_accuracy: 0.7752
Epoch 187/200
120/120 [==============================] - ETA: 0s - loss: 0.4702 - accuracy: 0.8416
Epoch 187: val_loss did not improve from 0.64304
120/120 [==============================] - 0s 4ms/step - loss: 0.4702 - accuracy: 0.8416 - val_loss: 0.6487 - val_accuracy: 0.7773
Epoch 188/200
116/120 [============================>.] - ETA: 0s - loss: 0.4553 - accuracy: 0.8556
Epoch 188: val_loss did not improve from 0.64304
120/120 [======

## k-fold Training evaluation

In [16]:
evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Test,0.739718,0.757655,0.810415,0.707795,0.771615,0.481568
Train,0.835643,0.856347,0.918553,0.806887,0.864397,0.672569


In [17]:
evaluations_df[evaluations_df["Train_Test"] == 'Test']

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
1,0,Test,0.725367,0.726891,"[0.0, 0.0041841004184100415, 0.071129707112970...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.999749, 0.999749, 0.9904192, 0.98997295, 0....",0.796755,0.723849,0.726891,0.450740
3,1,Test,0.746331,0.762332,"[0.0, 0.004201680672268907, 0.0546218487394958...","[0.0, 0.0, 0.0, 0.0041841004184100415, 0.00418...","[1.9977884, 0.9977884, 0.9898033, 0.9857631, 0...",0.801009,0.714286,0.778243,0.493571
5,2,Test,0.733193,0.751131,"[0.0, 0.004201680672268907, 0.0714285714285714...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9999557, 0.99995565, 0.9865463, 0.9865393, ...",0.808541,0.697479,0.768908,0.467581
7,3,Test,0.743697,0.798969,"[0.0, 0.004201680672268907, 0.0504201680672268...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9996699, 0.9996699, 0.99448156, 0.99418336,...",0.812937,0.651261,0.836134,0.495944
9,4,Test,0.750000,0.748954,"[0.0, 0.004201680672268907, 0.0420168067226890...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9999757, 0.9999757, 0.9991516, 0.9982273, 0...",0.832833,0.752101,0.747899,0.500004


In [18]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Train,0.849344,0.872340,"[0.0, 0.0010504201680672268, 0.031512605042016...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.9999899, 0.99998987, 0.99802667, 0.9979911,...",0.928331,0.818277,0.880378,0.700022
1,0,Test,0.725367,0.726891,"[0.0, 0.0041841004184100415, 0.071129707112970...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.999749, 0.999749, 0.9904192, 0.98997295, 0....",0.796755,0.723849,0.726891,0.450740
2,1,Train,0.829396,0.846578,"[0.0, 0.001049317943336831, 0.0136411332633788...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00105042...","[1.9999757, 0.9999757, 0.9973654, 0.99733377, ...",0.909746,0.804827,0.853992,0.659605
3,1,Test,0.746331,0.762332,"[0.0, 0.004201680672268907, 0.0546218487394958...","[0.0, 0.0, 0.0, 0.0041841004184100415, 0.00418...","[1.9977884, 0.9977884, 0.9898033, 0.9857631, 0...",0.801009,0.714286,0.778243,0.493571
4,2,Train,0.815845,0.827174,"[0.0, 0.001049317943336831, 0.0545645330535152...","[0.0, 0.0, 0.0, 0.001049317943336831, 0.001049...","[1.9991243, 0.9991242, 0.98641765, 0.9861864, ...",0.904854,0.798531,0.833158,0.632068
5,2,Test,0.733193,0.751131,"[0.0, 0.004201680672268907, 0.0714285714285714...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9999557, 0.99995565, 0.9865463, 0.9865393, ...",0.808541,0.697479,0.768908,0.467581
6,3,Train,0.830010,0.866123,"[0.0, 0.001049317943336831, 0.0451206715634837...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.00104931...","[1.9999998, 0.99999976, 0.99702615, 0.9970023,...",0.916085,0.780693,0.879328,0.663255
7,3,Test,0.743697,0.798969,"[0.0, 0.004201680672268907, 0.0504201680672268...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9996699, 0.9996699, 0.99448156, 0.99418336,...",0.812937,0.651261,0.836134,0.495944
8,4,Train,0.853620,0.869518,"[0.0, 0.001049317943336831, 0.0430220356768100...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[1.9999977, 0.99999774, 0.9981901, 0.99815446,...",0.933750,0.832109,0.875131,0.707896
9,4,Test,0.750000,0.748954,"[0.0, 0.004201680672268907, 0.0420168067226890...","[0.0, 0.0, 0.0, 0.004201680672268907, 0.004201...","[1.9999757, 0.9999757, 0.9991516, 0.9982273, 0...",0.832833,0.752101,0.747899,0.500004


# Independent data evaluation

## Using k-fold Models

### Performance of each k-fold model

In [19]:
## create the evaluation data structure for all iterations
evaluations = {
    "Fold" : [],
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    label_pred = pred2label(y_pred)

    # Compute precision, recall, sensitivity, specifity, mcc
    acc = accuracy_score(indpe_labels, label_pred)
    prec = precision_score(indpe_labels,label_pred)
    mcc = matthews_corrcoef(indpe_labels, label_pred)

    conf = confusion_matrix(indpe_labels, label_pred)
    tn, fp, fn, tp = conf.ravel()
    sens = tp/(tp+fn)
    spec = tn/(tn+fp)
    
    fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
    auc = roc_auc_score(indpe_labels, y_pred)

    evaluations["Fold"].append(i)
    evaluations["Train_Test"].append("Independent")
    evaluations["Accuracy"].append(acc)
    evaluations["Precision"].append(prec)
    evaluations["TPR"].append(tpr)
    evaluations["FPR"].append(fpr)
    evaluations["TPR_FPR_Thresholds"].append(thresholds)
    evaluations["AUC"].append(auc)
    evaluations["Sensitivity"].append(sens)
    evaluations["Specificity"].append(spec)
    evaluations["MCC"].append(mcc)
    
    del model
    tf.keras.backend.clear_session()

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.615837,0.236618,0.643466,0.592118,0.620548,0.160545


In [20]:
evaluations_df

,Fold,Train_Test,Accuracy,Precision,TPR,FPR,TPR_FPR_Thresholds,AUC,Sensitivity,Specificity,MCC
0,0,Independent,0.613878,0.242366,"[0.0, 0.0, 0.009852216748768473, 0.00985221674...","[0.0, 0.0009784735812133072, 0.000978473581213...","[1.9999899, 0.99998987, 0.9995154, 0.999508, 0...",0.654114,0.625616,0.611546,0.178235
1,1,Independent,0.604898,0.229287,"[0.0, 0.0, 0.014778325123152709, 0.01477832512...","[0.0, 0.0009784735812133072, 0.000978473581213...","[1.9999757, 0.9999757, 0.9988452, 0.99510694, ...",0.642474,0.586207,0.608611,0.146594
2,2,Independent,0.613878,0.237354,"[0.0, 0.0, 0.014778325123152709, 0.01477832512...","[0.0, 0.0009784735812133072, 0.000978473581213...","[1.9999557, 0.99995565, 0.9975981, 0.9972519, ...",0.639143,0.600985,0.616438,0.163819
3,3,Independent,0.630204,0.237395,"[0.0, 0.0, 0.029556650246305417, 0.02955665024...","[0.0, 0.0009784735812133072, 0.000978473581213...","[1.9999998, 0.99999976, 0.99844724, 0.9980725,...",0.642296,0.556650,0.644814,0.153684
4,4,Independent,0.616327,0.236686,"[0.0, 0.0, 0.009852216748768473, 0.00985221674...","[0.0, 0.0009784735812133072, 0.000978473581213...","[1.9999977, 0.99999774, 0.99992156, 0.99951375...",0.639302,0.591133,0.621331,0.160395


### Mean score with k-fold models

In [21]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    total_pred += y_pred
    all_preds.append(y_pred)
    
    del model
    tf.keras.backend.clear_session()
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.618776,0.237052,0.648822,0.586207,0.625245,0.159869


### Voting score with k-fold models

In [22]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

total_pred = np.zeros(indpe_labels.shape)
all_preds = []

for i in range(n_fold):
    
    current_model_path = os.path.join(modelPath, "bestModel-fold{}.hdf5".format(i))
    model = tf.keras.models.load_model(current_model_path)

    y_pred = model.predict(indpe_features)
    vote_pred = pred2label(y_pred)
    total_pred += vote_pred
    all_preds.append(vote_pred)
    
    del model
    tf.keras.backend.clear_session()
    
total_pred = total_pred / n_fold
label_pred = pred2label(total_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, total_pred)
auc = roc_auc_score(indpe_labels, total_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.62449,0.242485,0.630773,0.596059,0.630137,0.171175


## Using New Model

Train model on full data from training. Predict and evaluate on Independent data.

In [23]:
model = DLNN_Classifier(input_vec_shape = input_vec_shape)
    
## Define the model callbacks for early stopping and saving the model. Then train model
current_model_path = os.path.join(modelPath, "_fullModel.hdf5")
modelCallbacks = [
    tf.keras.callbacks.ModelCheckpoint(current_model_path,
                                       monitor = monitor, verbose = 1, save_best_only = True, 
                                       save_weights_only = False, mode = 'auto', save_freq = 'epoch'),
]

# adding random shuffling of the dataset for training purpose
index_arr = np.arange(train_features.shape[0])
index_arr = np.random.permutation(index_arr)

model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
          callbacks = modelCallbacks, validation_data = (indpe_features, indpe_labels))
# model.fit(x = train_features[index_arr], y = train_labels[index_arr], batch_size = batch_size, epochs = epochs, verbose = 1, 
#           callbacks = modelCallbacks, validation_split = 0.2)

model = tf.keras.models.load_model(current_model_path)

Epoch 1/200
141/149 [===========================>..] - ETA: 0s - loss: 1.0671 - accuracy: 0.5106
Epoch 1: val_loss improved from inf to 0.99425, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\_fullModel.hdf5
149/149 [==============================] - 1s 6ms/step - loss: 1.0642 - accuracy: 0.5118 - val_loss: 0.9943 - val_accuracy: 0.4384
Epoch 2/200
140/149 [===========================>..] - ETA: 0s - loss: 1.0260 - accuracy: 0.5308
Epoch 2: val_loss improved from 0.99425 to 0.95789, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\_fullModel.hdf5
149/149 [==============================] - 1s 5ms/step - loss: 1.0270 - accuracy: 0.5281 - val_loss: 0.9579 - val_accuracy: 0.5927
Epoch 3/200
140/149 [===========================>..] - ETA: 0s - loss: 1.0159 - accuracy: 0.5214
Epoch 3: val_loss improved from 0.95789 to 0.93536, saving model to Results\NT_Site_PredNTS_Classification_DLNN_KgapRFE\5fold\models\_fullModel.hdf5
149/

Epoch 27/200
149/149 [==============================] - ETA: 0s - loss: 0.7830 - accuracy: 0.7112
Epoch 27: val_loss did not improve from 0.90253
149/149 [==============================] - 1s 5ms/step - loss: 0.7830 - accuracy: 0.7112 - val_loss: 0.9204 - val_accuracy: 0.6351
Epoch 28/200
149/149 [==============================] - ETA: 0s - loss: 0.7749 - accuracy: 0.7292
Epoch 28: val_loss did not improve from 0.90253
149/149 [==============================] - 1s 4ms/step - loss: 0.7749 - accuracy: 0.7292 - val_loss: 0.9225 - val_accuracy: 0.6318
Epoch 29/200
149/149 [==============================] - ETA: 0s - loss: 0.7793 - accuracy: 0.7162
Epoch 29: val_loss did not improve from 0.90253
149/149 [==============================] - 1s 4ms/step - loss: 0.7793 - accuracy: 0.7162 - val_loss: 0.9166 - val_accuracy: 0.6367
Epoch 30/200
140/149 [===========================>..] - ETA: 0s - loss: 0.7781 - accuracy: 0.7281
Epoch 30: val_loss did not improve from 0.90253
149/149 [==============

Epoch 57/200
148/149 [============================>.] - ETA: 0s - loss: 0.6646 - accuracy: 0.7762
Epoch 57: val_loss did not improve from 0.90253
149/149 [==============================] - 1s 4ms/step - loss: 0.6648 - accuracy: 0.7758 - val_loss: 0.9482 - val_accuracy: 0.6139
Epoch 58/200
148/149 [============================>.] - ETA: 0s - loss: 0.6575 - accuracy: 0.7774
Epoch 58: val_loss did not improve from 0.90253
149/149 [==============================] - 1s 5ms/step - loss: 0.6587 - accuracy: 0.7767 - val_loss: 0.9450 - val_accuracy: 0.6180
Epoch 59/200
136/149 [==========================>...] - ETA: 0s - loss: 0.6554 - accuracy: 0.7776
Epoch 59: val_loss did not improve from 0.90253
149/149 [==============================] - 1s 4ms/step - loss: 0.6569 - accuracy: 0.7767 - val_loss: 0.9460 - val_accuracy: 0.6171
Epoch 60/200
144/149 [===========================>..] - ETA: 0s - loss: 0.6574 - accuracy: 0.7730
Epoch 60: val_loss did not improve from 0.90253
149/149 [==============

Epoch 87/200
148/149 [============================>.] - ETA: 0s - loss: 0.5988 - accuracy: 0.7897
Epoch 87: val_loss did not improve from 0.90253
149/149 [==============================] - 1s 5ms/step - loss: 0.5987 - accuracy: 0.7893 - val_loss: 0.9421 - val_accuracy: 0.6310
Epoch 88/200
148/149 [============================>.] - ETA: 0s - loss: 0.5991 - accuracy: 0.7880
Epoch 88: val_loss did not improve from 0.90253
149/149 [==============================] - 1s 5ms/step - loss: 0.6013 - accuracy: 0.7880 - val_loss: 0.9530 - val_accuracy: 0.6269
Epoch 89/200
148/149 [============================>.] - ETA: 0s - loss: 0.5893 - accuracy: 0.7901
Epoch 89: val_loss did not improve from 0.90253
149/149 [==============================] - 1s 5ms/step - loss: 0.5893 - accuracy: 0.7909 - val_loss: 0.9554 - val_accuracy: 0.6327
Epoch 90/200
146/149 [============================>.] - ETA: 0s - loss: 0.6032 - accuracy: 0.7872
Epoch 90: val_loss did not improve from 0.90253
149/149 [==============

149/149 [==============================] - 1s 5ms/step - loss: 0.5455 - accuracy: 0.8090 - val_loss: 0.9580 - val_accuracy: 0.6335
Epoch 117/200
149/149 [==============================] - ETA: 0s - loss: 0.5378 - accuracy: 0.8098
Epoch 117: val_loss did not improve from 0.90253
149/149 [==============================] - 1s 4ms/step - loss: 0.5378 - accuracy: 0.8098 - val_loss: 0.9816 - val_accuracy: 0.6335
Epoch 118/200
140/149 [===========================>..] - ETA: 0s - loss: 0.5364 - accuracy: 0.8121
Epoch 118: val_loss did not improve from 0.90253
149/149 [==============================] - 1s 5ms/step - loss: 0.5340 - accuracy: 0.8140 - val_loss: 0.9680 - val_accuracy: 0.6351
Epoch 119/200
144/149 [===========================>..] - ETA: 0s - loss: 0.5369 - accuracy: 0.8069
Epoch 119: val_loss did not improve from 0.90253
149/149 [==============================] - 1s 5ms/step - loss: 0.5377 - accuracy: 0.8060 - val_loss: 0.9837 - val_accuracy: 0.6294
Epoch 120/200
146/149 [=========

Epoch 146/200
135/149 [==========================>...] - ETA: 0s - loss: 0.4893 - accuracy: 0.8250
Epoch 146: val_loss did not improve from 0.90253
149/149 [==============================] - 1s 4ms/step - loss: 0.4819 - accuracy: 0.8300 - val_loss: 1.0304 - val_accuracy: 0.6392
Epoch 147/200
135/149 [==========================>...] - ETA: 0s - loss: 0.4872 - accuracy: 0.8292
Epoch 147: val_loss did not improve from 0.90253
149/149 [==============================] - 1s 4ms/step - loss: 0.4839 - accuracy: 0.8346 - val_loss: 1.0148 - val_accuracy: 0.6424
Epoch 148/200
145/149 [============================>.] - ETA: 0s - loss: 0.4888 - accuracy: 0.8315
Epoch 148: val_loss did not improve from 0.90253
149/149 [==============================] - 1s 5ms/step - loss: 0.4901 - accuracy: 0.8296 - val_loss: 0.9997 - val_accuracy: 0.6449
Epoch 149/200
148/149 [============================>.] - ETA: 0s - loss: 0.4815 - accuracy: 0.8383
Epoch 149: val_loss did not improve from 0.90253
149/149 [======

149/149 [==============================] - 1s 5ms/step - loss: 0.4459 - accuracy: 0.8585 - val_loss: 1.0755 - val_accuracy: 0.6506
Epoch 176/200
136/149 [==========================>...] - ETA: 0s - loss: 0.4408 - accuracy: 0.8516
Epoch 176: val_loss did not improve from 0.90253
149/149 [==============================] - 1s 4ms/step - loss: 0.4426 - accuracy: 0.8522 - val_loss: 1.0710 - val_accuracy: 0.6522
Epoch 177/200
137/149 [==========================>...] - ETA: 0s - loss: 0.4382 - accuracy: 0.8618
Epoch 177: val_loss did not improve from 0.90253
149/149 [==============================] - 1s 4ms/step - loss: 0.4392 - accuracy: 0.8615 - val_loss: 1.0872 - val_accuracy: 0.6482
Epoch 178/200
138/149 [==========================>...] - ETA: 0s - loss: 0.4319 - accuracy: 0.8614
Epoch 178: val_loss did not improve from 0.90253
149/149 [==============================] - 1s 4ms/step - loss: 0.4292 - accuracy: 0.8623 - val_loss: 1.0884 - val_accuracy: 0.6490
Epoch 179/200
138/149 [=========

In [24]:
## create the evaluation data structure for all iterations
evaluations = {
    "Train_Test" : [],
    "Accuracy" : [],
    "Precision": [],
    "TPR": [],
    "FPR": [],
    "TPR_FPR_Thresholds": [],
    "AUC": [],
    "Sensitivity": [],
    "Specificity": [],
    "MCC":[]
}

##################################################################################
##### Prediction and metrics for Independent dataset
##################################################################################

y_pred = model.predict(indpe_features)
label_pred = pred2label(y_pred)

# Compute precision, recall, sensitivity, specifity, mcc
acc = accuracy_score(indpe_labels, label_pred)
prec = precision_score(indpe_labels,label_pred)
mcc = matthews_corrcoef(indpe_labels, label_pred)

conf = confusion_matrix(indpe_labels, label_pred)
tn, fp, fn, tp = conf.ravel()
sens = tp/(tp+fn)
spec = tn/(tn+fp)

fpr, tpr, thresholds = roc_curve(indpe_labels, y_pred)
auc = roc_auc_score(indpe_labels, y_pred)

evaluations["Train_Test"].append("Independent")
evaluations["Accuracy"].append(acc)
evaluations["Precision"].append(prec)
evaluations["TPR"].append(tpr)
evaluations["FPR"].append(fpr)
evaluations["TPR_FPR_Thresholds"].append(thresholds)
evaluations["AUC"].append(auc)
evaluations["Sensitivity"].append(sens)
evaluations["Specificity"].append(spec)
evaluations["MCC"].append(mcc)

##################################################################################

evaluations_df = pd.DataFrame.from_dict(evaluations)

evaluations_df_grouped = evaluations_df.groupby(["Train_Test"]).mean().filter(['Accuracy', 
                                                                               'Precision', 
                                                                               'AUC', 
                                                                               'Sensitivity', 
                                                                               'Specificity', 
                                                                               'MCC'])

evaluations_df_grouped

,Accuracy,Precision,AUC,Sensitivity,Specificity,MCC
Train_Test,,,,,,
Independent,0.656327,0.247685,0.628192,0.527094,0.681996,0.162715


In [25]:
print(classification_report(indpe_labels, np.round(y_pred).astype(int)))

              precision    recall  f1-score   support

           0       0.88      0.68      0.77      1022
           1       0.25      0.53      0.34       203

    accuracy                           0.66      1225
   macro avg       0.56      0.60      0.55      1225
weighted avg       0.77      0.66      0.70      1225

